In [1]:
from pennystockpipeline.PennyStockData import PennyStockData
import numpy as np
import pandas as pd
from time import gmtime, strftime
from datetime import datetime, timedelta
import torch

torch.manual_seed(0)

## Initializing
DATABASE_NAME_WITH_PATH = "mod_historicaldata.db"
TABLE_NAME = "all_historical_modified"

#psd = PennyStockData(database_name_with_path, table_name, impute=True, verbose=2).get_columns(columns).normalize(['close', 'volume_weighted_average']).create_sequences(sequence_length, prediction_length)
psd = PennyStockData(DATABASE_NAME_WITH_PATH, TABLE_NAME, impute=True, verbose=2)


[1, 'ACHR', '2022-07-12', '16:30', 3.0307, 3.0301, 3.03, 3.0301, 3.03, 1657643400, 269, 4]
[DEBUG][PennyStockData]: Imputed len(data): 965607
[1, 'ACHR', '2022-07-12', '16:30', 3.0307, 3.0301, 3.03, 3.0301, 3.03, 1657643400, 269, 4, 0]
next_max_date: 2024-06-01


In [2]:
#columns = ['ticker_id', 'p_date', 'close', 'volume_weighted_average']
COLUMNS = ['ticker_id', 'p_date', 'p_time', 'volume_weighted_average']
COLUMNS_TO_NORMALIZE = ['volume_weighted_average']

SEQUENCE_LENGTH = 20
PREDICTION_LENGTH = 20

TRAIN_TEST_SPLIT = 0.8
psd = psd.get_columns(COLUMNS).normalize(COLUMNS_TO_NORMALIZE)#.create_sequences(SEQUENCE_LENGTH, PREDICTION_LENGTH).split_dataset(split=TRAIN_TEST_SPLIT, to_torch=True)


[INFO][PennyStockData]: Performing ticker-wise normalization on ['volume_weighted_average']
[1, '2022-07-12', '16:30', 3.0307]
[INFO][PennyStockData]: Performing global normalization on ['volume_weighted_average'] using MixMaxScaler
[0.3889893316214954]


In [3]:
# Library for implementing Deep Neural Network 
from pennystockpipeline.PennyStockLabeller import PennyStockLabeller
HORIZON = 1.01
COLUMNS_TO_COMPARE = ["volume_weighted_average"]

labeller = PennyStockLabeller(psd, 2)
#labeller = labeller.labelize(column_to_compare = COLUMN_TO_COMPARE, horizon = HORIZON)


In [ ]:
print(np.unique(np.transpose(psd.data)[0]))

df = pd.DataFrame(psd.data, columns=psd.headers)
#df.agg(x=('A', 'max'), y=('B', 'min'), z=('C', 'mean'))
df.groupby('ticker_id').count()


In [ ]:
'''
def labelize(labeller, columns_to_compare=["volume_weighted_average"], horizon = 0.1):
    unlabelled_data_raw = np.array(psd.data)
    number_of_records = len(unlabelled_data_raw)
    
    unlabelled_data_df = pd.DataFrame(unlabelled_data_raw, columns=psd.headers)
    
    columns_to_compare_str = [column_to_compare + "_h_str" for column_to_compare in columns_to_compare]
    labels_df_data = ["no_buy"] * number_of_records
    labels_df = pd.DataFrame(data = labels_df_data, columns = columns_to_compare_str)

    labels_cursor = 0

    for ticker_id in unlabelled_data_df['ticker_id'].unique():
        ticker_data_df = unlabelled_data_df[unlabelled_data_df['ticker_id'] == ticker_id].copy()
        print(f'Ticker: {ticker_id}')
        
        for ticker_date in ticker_data_df['p_date'].unique():
            ticker_date_data_df = ticker_data_df[ticker_data_df['p_date'] == ticker_date].copy()

            print(f'---> Date: {ticker_date}')
            #ticker_date_data_df.reset_index(drop=True, inplace=True)
            #ticker_date_data_df.reindex()

            buy_triggered = False
            #print(f'label start: {labels_cursor}')
            #labels_cursor = labels_cursor + len(ticker_date_data_df)
            #print(f'label end: {labels_cursor}')

            labels_df_cursor = 0

            for ctc in columns_to_compare:
                if ctc in ticker_date_data_df.columns:
                    for m in ticker_date_data_df.index:
                        print(f'---> ticker_date_data_df.index.min() and max(): {ticker_date_data_df.index.min()}, {ticker_date_data_df.index.max()}')
                        if m >= ticker_date_data_df.index.max():
                            break

                        #print(f'{(np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m]))}')
                        #print(f'{horizon < (np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m]))}')
                        #x = lambda a : a + 10
                        comparison_lambda = lambda h, x, d: np.insert((h < (np.float32(x) / np.float32(d))), 0, False)
                        comparison_truth_table = comparison_lambda(horizon, ticker_date_data_df.loc[m+1:, ctc], ticker_date_data_df.loc[m, ctc])
                        #comparison_truth_table = (np.insert((horizon < (np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m]))), 0, False))
                        print(f'--------> {m} comparison_truth_table: {comparison_truth_table}')
                        
                        #print(["buy" if h else "no_buy" for h in (np.insert((horizon < (np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m]))), 0, False)).reshape(-1,1).squeeze()])
                        #another_array = np.array(["buy" if h else "no_buy" for h in (np.insert(horizon < (np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m])), 0, False))])
                        #print(f'--------> {another_array.shape} another_array: {another_array}')
                        
                        #if True in comparison_truth_table:
                        ### Now update the df _h_str and break for this day
                        # Getting only true items
                        labels_df_indices = ticker_date_data_df[:len(comparison_truth_table)].index[comparison_truth_table]
                        #print(f'labels_df_indices: {labels_df_indices}')
                        #labels_df[ctc+"_h_str"][labels_df_indices] = "buy"
                        labels_df.loc[labels_df_indices, ctc+"_h_str"] = "buy"
                        print(labels_df[ctc+"_h_str"][ticker_date_data_df.index])
                        #labels_df.loc[ticker_date_data_df.index[m]:-1, ctc+"_h_str"] = ["buy" if h else "no_buy" for h in (np.insert(horizon < (np.float32(ticker_date_data_df[ctc][m+1:]) / np.float32(ticker_date_data_df[ctc][m])), 0, False))].reshape(-1,1).squeeze()
                        if (labels_df_indices.shape[0] > 0):
                            buy_triggered = True
                            ticker_date_data_df = None
                            break
                            
                        
                    if buy_triggered:
                        ticker_date_data_df = None
                        break
        ticker_date_data_df = None
        
    labelled_data_df = pd.concat([unlabelled_data_df, labels_df], axis=1)        
    labeller.labelled_data_df = labelled_data_df

    return labeller

HORIZON = 1.001
lbller = labelize(labeller, columns_to_compare = COLUMNS_TO_COMPARE, horizon = HORIZON)
'''

In [ ]:
labeller = labeller.labelize(columns_to_compare = COLUMNS_TO_COMPARE, horizon = HORIZON)



In [5]:
labeller.to_csv('labelled_psd_data.csv')
#lbller.labelled_data_df.to_csv('labelled_psd.csv')

In [ ]:
'''
data_by_ticker = {}
    for ticker in unlabelled_data_df['ticker_id'].unique():
        data_by_ticker[ticker] = unlabelled_data_df[unlabelled_data_df['ticker_id'] == ticker].copy()
        # now, we loop through each date and time to find if vwa > ticker's_first_vwa * gain_threshold * 100  first value
        for ticker_date in data_by_ticker[ticker]['p_date'].unique():
            data_by_ticker_date = np.indices(data_by_ticker[ticker]['p_date'] == ticker_date)
            
            data_by_ticker_date = pd.DataFrame([data_by_ticker[ticker]['p_date'] == ticker_date], columns=data_by_ticker.keys())
            print(data_by_ticker_date)
            
            print(data_by_ticker[ticker]['p_date'] == ticker_date)
            print(f'ticker: {ticker} date: {ticker_date}')

            #print(data_by_ticker[data_by_ticker[ticker]['p_date'] == ticker_date])

            
            #data_by_ticker_date[ticker_date] = data_by_ticker[ticker][data_by_ticker[ticker]['p_date'] == ticker_date].copy()
            for ctn in column_to_compare:
                new_column = []
                if ctn in unlabelled_data_df.columns:
                    #print(len(data_by_ticker[data_by_ticker[ticker]['p_date']==ticker_date]))
                    #data_by_ticker_date[ticker_date][ctn+"_h_str"] = 'no_buy'
                    #(data_by_ticker[ticker]['p_date'] == ticker_date).ctn
                    #h_values = data_by_ticker[data_by_ticker[ticker][ticker_date]][ctn] / data_by_ticker[data_by_ticker[ticker][ticker_date]][ctn][0]
                    h_values = (data_by_ticker[ticker]['p_date'] == ticker_date)[ctn] / (data_by_ticker[ticker]['p_date'] == ticker_date)[ctn][0]
                    new_column[ctn+"_h_str"] = ["buy" if np.greater_equal(h_values, horizon) else 'no_buy']
                    ## new_column[ctn+"_h_str"] = ["buy" if np.any(h_values, where=np.greater_equal(horizon)) else 'no_buy']
                    data_by_ticker[ticker] = pd.concat(data_by_ticker[ticker], new_column[ctn+"_h_str"])
                    ##data_by_ticker[ticker][data_by_ticker[ticker]['p_date'] == ticker_date]

            
            #data_by_ticker_date[ticker_date] = pd.DataFrame()
            #data_by_ticker_date[ticker_date] = data_by_ticker[ticker][data_by_ticker[ticker]['p_date'] == ticker_date].copy()
            # each record is divided by the first record, i.e. vwa at day start
            #for ctn in column_to_compare:
            #    if ctn in unlabelled_data_df.columns:
            #        # we initialize the h_values with 'no buy'
            #        data_by_ticker_date[ticker_date][ctn+"_h_str"] = 'no_buy'
            #        h_values = data_by_ticker_date[ticker_date][ctn] / data_by_ticker_date[ticker_date][ctn][0]
            #        #data_by_ticker[ticker][ctn+"_h_str"] = ["buy" if h >= horizon for h in h_values]
            #        data_by_ticker_date[ticker_date][ctn+"_h_str"] = ["buy" if np.any(h_values, where=np.greater_equal(horizon)) else 'no_buy']

        #for ticker_date in data_by_ticker[ticker]:
            # create a temporary DataFrame to hold the current data
        temp_df = pd.DataFrame(data_by_ticker[ticker].values, columns=data_by_ticker[ticker].keys())
        labelled_data = pd.concat([labelled_data, temp_df], axis=0, ignore_index=True)
        
    #for ticker in data_by_ticker:
    #    # create a temporary DataFrame to hold the current data
    #    temp_df = pd.DataFrame(labelled_data.values, columns=labelled_data.keys())
    #    labelled_data = pd.concat([labelled_data, temp_df], axis=0, ignore_index=True)

    # optionally, you can reset the index if needed
    labelled_data.reset_index(drop=True, inplace=True)

    #labelled_data[column_to_compare] = labeller.scaler.fit_transform(labelled_data[column_to_compare])
    
    #normalized_data = normalized_data.drop(columns=['ticker_id'])
    #normalized_data.reset_index(drop=True, inplace=True)

    print(f'Length of labelled_data: {len(labelled_data)}')
    print(f'labelled_data:')
    print(f'{labelled_data}')

    labeller.labelled_data = labelled_data.values.tolist()
    labeller.labelled_headers = labelled_data.columns.tolist()
    
    labeller.label_data = labelled_data["g_value"].values.tolist()
    labeller.label_header = "g_value"

'''